In [ ]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error

%matplotlib inline

In [ ]:
df = pd.read_csv("/kaggle/input/california-housing-prices/housing.csv")

# Basic Data Exploration

In [ ]:
df.head()

In [ ]:
df['median_house_value'].hist(bins=40)

In [ ]:
df.shape

In [ ]:
ax = df.hist(figsize=[20, 18], bins = 50)

In [ ]:
ax = pd.plotting.scatter_matrix(df, alpha=0.2, figsize = [20, 20])

# Creating New Features 

In [ ]:
df['peopleperrooms'] = df['households']/df['total_rooms']

In [ ]:
df['latlon'] = df['longitude'] * df['latitude']

In [ ]:
df['roomsperbedrooms'] = df['total_rooms']/df['total_bedrooms']

In [ ]:
df['peopleperbedrooms'] = df['households']/df['total_bedrooms']

In [ ]:
df['ageperpop'] = df['housing_median_age']/df['households']

In [ ]:
df['ageperbeds'] = df['housing_median_age']/df['total_bedrooms']

In [ ]:
list(df.columns.values)

In [ ]:
df.head()

In [ ]:
ax = pd.plotting.scatter_matrix(df[['median_house_value', 
                                    'peopleperrooms', 
                                    'latlon', 
                                    'roomsperbedrooms',
                                    'peopleperbedrooms',
                                    'ageperpop',
                                    'ageperbeds']], figsize = [32, 28])

## Remove null data

In [ ]:
df[df.isnull().any(axis=1)]

In [ ]:
df = df[~df.isnull().any(axis=1)]

# Create clustering for lat and long

In [ ]:
plt.scatter(df['longitude'], df['latitude'], marker = ".")

In [ ]:
X = df[['longitude','latitude']]
means = KMeans(n_clusters=60, random_state=0).fit(X)

In [ ]:
means.cluster_centers_[0:5]

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
ax.scatter(df['longitude'], df['latitude'], marker = ".")
centroids = means.cluster_centers_

ax.scatter(centroids[:, 0], centroids[:, 1], color = "black", marker = ",")

In [ ]:
df['geo_cluster'] = means.predict(X)

In [ ]:
df.groupby("geo_cluster")['median_house_value'].mean().head()

In [ ]:
df.groupby("geo_cluster")['median_house_value'].mean().sort_values().head()

- Let's order the clusters centers by housing value and use this as a feature.

In [ ]:
index = df.groupby("geo_cluster")['median_house_value'].mean().sort_values().index
new_index = range(0, 60)
dict_index = dict(zip(index, new_index))
dict_index

In [ ]:
df['geo_cluster_ordered'] = df['geo_cluster'].apply(lambda x: dict_index[x])

In [ ]:
df.head()

# Transform ocean_proximity into a numerical feature

In [ ]:
df.groupby('ocean_proximity')['median_house_value'].mean()

In [ ]:
values = df.groupby('ocean_proximity')['median_house_value'].mean()
new_values = dict(zip(values.index, values.values))
new_values

In [ ]:
df['n_ocean_proximity'] = df['ocean_proximity'].apply(lambda x: new_values[x])

In [ ]:
ax = pd.plotting.scatter_matrix(df[['median_house_value', 'n_ocean_proximity', 'geo_cluster_ordered']], alpha=0.8, figsize = [40, 36])

# Split Dataset Into Training and Test

In [ ]:
X = df.drop(columns=['median_house_value', 'ocean_proximity', 'latlon', 'geo_cluster'])
y = df['median_house_value']

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Prediction

In [ ]:
model = GradientBoostingRegressor().fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

# Results

In [ ]:
print(np.sqrt(mean_squared_error(y_test,y_pred)))

In [ ]:
r2_score(y_test, y_pred)

In [ ]:
mean_absolute_error(y_test, y_pred)